<a href="https://colab.research.google.com/github/fidan-c/human-value-detection/blob/main/early_bird.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Only with Premise: bert-base-uncased

!pip install datasets evaluate transformers[sentencepiece]
!pip install transformers
!pip install tokenizer
!pip install simpletransformers

In [ ]:

# tokenizer and dynamic padding imports
import tensorflow as tf
from transformers import AutoTokenizer,TFAutoModel, DataCollatorWithPadding
import numpy as np
import pandas as pd


from datasets import load_dataset  # was necessary for using Hugging Face transformers to have a dictonary, not needed for "simple transformers".

In [ ]:
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)

In [ ]:
# load data - not including chinese
# sep for tsv
arguments_training_no_label = pd.read_csv('/content/argument-data/arguments-training.tsv', sep='\t')
arguments_validation_no_label = pd.read_csv('/content/argument-data/arguments-validation.tsv', sep='\t')
arguments_test_data = pd.read_table('/content/argument-data/arguments-test.tsv')

labels_training = pd.read_csv('/content/argument-data/labels-training.tsv', sep='\t').drop('Argument ID', axis=1)
labels_validation= pd.read_csv('/content/argument-data/labels-validation.tsv', sep='\t').drop('Argument ID', axis=1)



# train_df = pd.concat([arguments_training_no_label, labels_training], axis=1)

# val_df= pd.concat([arguments_validation_no_label, labels_validation], axis=1)

train_labels = labels_training.values.tolist()
train_text = arguments_training_no_label['Premise'].values
train_data = [[t, l] for t, l in zip(train_text, train_labels)]  # see the format needed ["text",[label]]
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]

# repeat above for the validation set
val_labels = labels_validation.values.tolist()
val_text = arguments_validation_no_label['Premise'].values
val_data = [[t, l] for t, l in zip(val_text, val_labels)] 
val_df = pd.DataFrame(val_data)
val_df.columns = ["text", "labels"]

test_data = arguments_test_data['Premise'].values
np.shape(test_data)

(1576,)

In [ ]:
model_args = MultiLabelClassificationArgs(num_train_epochs=4)

# Create a MultiLabelClassificationModel from simple transformers: <https://simpletransformers.ai/docs/multi-label-classification/>
model = MultiLabelClassificationModel(
    "bert",
    "bert-base-uncased",
    num_labels=len(labels_training.columns),
    args=model_args,
)

# Train the model
model.train_model(train_df)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were not 

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

...

Running Epoch 0 of 4:   0%|          | 0/675 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/675 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/675 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/675 [00:00<?, ?it/s]

(2700, 0.3058816251820988)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(val_df)

  0%|          | 0/1896 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/237 [00:00<?, ?it/s]

In [ ]:
result

{'LRAP': 0.7035289251750706, 'eval_loss': 0.3318530461959195}

In [ ]:
test_preds = model.predict(list(test_data))

  0%|          | 0/1576 [00:00<?, ?it/s]

  0%|          | 0/197 [00:00<?, ?it/s]

In [ ]:
cols = ["Argument ID"] + list(labels_validation.columns)
pred_data = []

for arg_id, labels in zip(arguments_test_data["Argument ID"].values, test_preds[0]):
  row = [arg_id] + labels[:]
  pred_data.append(row)

pred_df = pd.DataFrame(pred_data, columns=cols)
pred_df.to_csv("test.tsv", index=False, sep='\t')